<a href="https://colab.research.google.com/github/s-bose/A-December-of-Algorithms-2019/blob/master/test_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Task 1.1 Write a function in python that identify which columns have date in them**

In [520]:
import numpy as np
import pandas as pd
import re

In [614]:
def get_date_cols(df, remove_nan=True):

  """
  Returns column names which has date values
  Converts the string date entries in the input dataframe into datetime64 object
  Input: 'df' - any pandas.DataFrame object
        'remove_nan' = (default) True, removes rows which have NaT value in the date column(s)
  After conversion, the format is YYYY-MM-DD, regardless of the original format      
  Invalid dates get converted to NaT and the corresponding rows are filtered out by default    
  """

  # regex string pattern to match strings with a date format

  reg = re.compile(
      r"""\s*  # optional whitespace
      (\d+)    # Day
      \s*
      [-/,.]   # separator
      \s*
      (\d+)    # Month
      \s*
      [-/,.]   # separator
      \s*
      (\d+)    # Years
      \s*      # optional whitespace
      """, re.VERBOSE)

  # first select all the nnon-numeric columns
  str_cols = df.select_dtypes('object').columns.tolist()

  # iterate over those column entries to filter out any column where no entry matches the regex

  for col in str_cols:
    if not df.loc[:, col].astype(str).str.match(reg).any():
      str_cols.remove(col)

  # additionally, remove whitespaces from the string date entries
  # and convert them to datetime64 objects

  for cols in str_cols: 
    df.loc[:, cols].replace(reg, r"\1/\2/\3", inplace=True)
    df.loc[:, cols] = df.loc[:, cols].apply(pd.to_datetime, dayfirst=True, utc=False, errors="coerce")

  if remove_nan:
    df.dropna(subset=str_cols, inplace=True)

  return str_cols


In [567]:
np.random.seed(42)

d = dict( A = np.random.randint(100, size=10), B = np.random.randn(10) , 
         C = np.array(['   32- 01-2020',
                         '  15  / 07/2020  ',
                         '12-31-1965    ',
                         '  17/02/  2020',
                         ' 2020/21/12',
                         ' 31 / 12 / 1996 ',
                         '19- 01-1999',
                         '',
                         ' 15-04-1996'
                         ]),
          D = np.array(['Alex', 'Brad', 'James', 'David', 'Dorothy', 'Sysadmin', 'Megan', 'Karen', 'Maverick'])
          )

date_df = pd.DataFrame(dict([ (key,pd.Series(vals)) for key, vals in d.items() ]))
date_df

,A,B,C,D
0,51,0.279041,32- 01-2020,Alex
1,92,1.010515,15 / 07/2020,Brad
2,14,-0.580878,12-31-1965,James
3,71,-0.525170,17/02/ 2020,David
4,60,-0.571380,2020/21/12,Dorothy
5,20,-0.924083,31 / 12 / 1996,Sysadmin
6,82,-2.612549,19- 01-1999,Megan
7,86,0.950370,,Karen
8,74,0.816445,15-04-1996,Maverick
9,74,-1.523876,NaN,NaN


In [568]:
date_cols = get_date_cols(date_df, remove_nan=False)

print(f'the columns with dates are: {date_cols}')

the columns with dates are: ['C']


In [569]:
date_df

,A,B,C,D
0,51,0.279041,NaT,Alex
1,92,1.010515,2020-07-15,Brad
2,14,-0.580878,1965-12-31,James
3,71,-0.525170,2020-02-17,David
4,60,-0.571380,2020-12-21,Dorothy
5,20,-0.924083,1996-12-31,Sysadmin
6,82,-2.612549,1999-01-19,Megan
7,86,0.950370,NaT,Karen
8,74,0.816445,1996-04-15,Maverick
9,74,-1.523876,NaT,NaN


**Task 1.2 Using these date columns make new columns which are difference between these columns taking 2 at a time**

In [612]:
# create a new dataframe

dates = ['31-08-1996', '01-12-1987', '15-12-1982', '1975-05-21', '1978-28-02']

data = dict(name = ['alex', 'john', 'david', 'maria', 'fischer'],
        work_hours_pw = np.random.randint(30, size=5),
        birthday = dates
        )


new_df = pd.DataFrame(data)
get_date_cols(new_df)

new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           5 non-null      object        
 1   work_hours_pw  5 non-null      int64         
 2   birthday       5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 160.0+ bytes


**Task 1.3 Drop all the original columns containing the date and just keep the newly computed columns**

In [613]:
new_df2 = pd.read_csv("demo.csv")
get_date_cols(new_df2)
new_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        5 non-null      object        
 1   work_hours  5 non-null      int64         
 2   birthday    5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 160.0+ bytes


In [630]:


# initialise data of lists. 
birth_date = ['31-08-1996', '01-12-1987', '15-12-1982', '1975-05-21', '1978-28-02']
start_date = ['01-01-2020', '05-17-2018', '31-08-2018', '2019-05-04', '11-12-2017']
end_date = ['01-01-2022', '31-12-2020', '01-09-2021', '05-06-2021', '01-01-2019']

data = {'name' : ['alex', 'john', 'david', 'maria', 'fischer'],
        'work_hours_pw' : np.random.randint(30, size=5),
        'birthday' : birth_date,
        'start_date' : start_date,
        'end_date' : end_date,       
      } 

df2 = pd.DataFrame(data)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           5 non-null      object
 1   work_hours_pw  5 non-null      int64 
 2   birthday       5 non-null      object
 3   start_date     5 non-null      object
 4   end_date       5 non-null      object
dtypes: int64(1), object(4)
memory usage: 328.0+ bytes


In [631]:
cols = get_date_cols(df2)
cols

['birthday', 'start_date', 'end_date']

In [633]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           5 non-null      object        
 1   work_hours_pw  5 non-null      int64         
 2   birthday       5 non-null      datetime64[ns]
 3   start_date     5 non-null      datetime64[ns]
 4   end_date       5 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(1)
memory usage: 240.0+ bytes
